In [ ]:
import random

import numpy as np

from dt_class import DecisionTree

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
qqq = np.array([[11, 12, 13, 14, 15], 
                [21, None, 23, 24, 25], 
                [31, 32, 33, 34, 35], 
                [41, 42, 43, 44, 45], 
                [51, 52, 53, 54, 55]]).astype(np.float64)
qqq

In [ ]:
for v in qqq[:, 2]:
    print(type(v))

In [ ]:
RANDOM_STATE = 14

In [ ]:
def insert_null_values(X, feat_count, null_ratio):
    for feat in range(feat_count):
        null_indexes = random.sample(range(X.shape[0]), round(X.shape[0] * null_ratio))
        X[:, feat][null_indexes] = np.nan

In [ ]:
tree = DecisionTree(criterion='entropy', verbose=True)

In [ ]:
X, y = make_classification(n_features=30, n_redundant=5, n_samples=100, random_state=RANDOM_STATE)

# insert null values
insert_null_values(X, 5, 0.2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
%%time

tree.fit(X_train, y_train)

In [ ]:
#%prun -l dt_class.py tree.fit(X_train, y_train)
#%prun tree.fit(X_train, y_train)

In [ ]:
tree.predict(X_test)[:10]

In [ ]:
tree.predict_proba(X_test)[:10]

In [ ]:
round(roc_auc_score(y_test, tree.predict_proba(X_test)[:, 1]), 2)

In [ ]:
round(roc_auc_score(y_train, tree.predict_proba(X_train)[:, 1]), 2)